Lineare Optimierung “Andere“

https://www.linkedin.com/feed/update/urn:li:activity:7350506543697543172/?midToken=AQHuucnYWQcIEA&midSig=2gKBl76uwTSXQ1&trk=eml-email_groups_recomm



🟢 𝗦𝗰𝗶𝗣𝘆 (𝘀𝗰𝗶𝗽𝘆.𝗼𝗽𝘁𝗶𝗺𝗶𝘇𝗲) - 𝟭𝟯.𝟭𝗸 ⭐
SciPy’s optimization module includes constrained and unconstrained optimization like linear and nonlinear programming.

A mature, versatile choice, widely adopted in science and engineering.

🟢 𝗚𝗼𝗼𝗴𝗹𝗲 𝗢𝗥-𝗧𝗼𝗼𝗹𝘀 - 𝟭𝟭.𝟮𝗸 ⭐
Ideal for combinatorial optimization (routing, scheduling), supporting linear and integer programming.

Highly efficient for complex operational research tasks.

🟢 𝗢𝗽𝘁𝘂𝗻𝗮 - 𝟭𝟬.𝟳𝗸 ⭐
Optuna offers Bayesian optimization and pruning for solving complex tasks, including multi-objective optimization.

Its flexible, define-by-run API adapts to various scenarios.

🟢 𝗛𝘆𝗽𝗲𝗿𝗢𝗽𝘁 - 𝟳.𝟮𝗸 ⭐
Known for TPE and Random Search, HyperOpt excels in resource scheduling and parameter selection.

Scales across machines, optimizing large datasets effectively.

🟢 𝗖𝗩𝗫𝗣𝗬 - 𝟱.𝟰𝗸 ⭐
CVXPY is a modeling library for convex optimization, popular in machine learning and economics.

Intuitive syntax makes complex convex problems easy to solve.

🟢 𝗖𝗩𝗫𝗣𝗬 𝗟𝗮𝘆𝗲𝗿𝘀 - 𝟭.𝟴𝗸 ⭐
An extension of CVXPY that embeds optimization layers in deep learning models, ideal for structured prediction and control within neural networks.

Works with PyTorch and TensorFlow.

🟢 𝗣𝘂𝗟𝗣 - 𝟮.𝟭𝗸 ⭐
A user-friendly library for linear and integer programming, suitable for prototyping, resource allocation, and scheduling.

Effective for small to medium LP problems.

🟢 𝗣𝘆𝗼𝗺𝗼 - 𝟮𝗸 ⭐
Flexible for large-scale optimization, supporting linear, nonlinear, and mixed-integer programming.

Used widely in energy systems and logistics, with strong multi-stage problem support.

🟢 𝗖𝗮𝘀𝗔𝗗𝗶 - 𝟭.𝟳𝗸 ⭐
A framework for dynamic optimization, used in engineering for real-time control.

Efficient handling of symbolic expressions makes it ideal for scientific computing.

🟢 𝗣𝘆𝗚𝗔𝗗 - 𝟭.𝟳𝗸 ⭐
A Genetic Algorithm library for combinatorial/discrete optimization.


🟢 𝗣𝘆𝗚𝗔𝗗 - 𝟭.𝟳𝗸 ⭐

https://web.casadi.org/

PyGAD is an open-source Python library for building the genetic algorithm and optimizing machine learning algorithms. It works with Keras and PyTorch.

In [1]:
import pygad
print(pygad.__version__)

3.5.0


In [3]:
import pygad
import numpy as np

# Objective: maximize profit → fitness = 182 * x + 139 * y
def fitness_func(ga_instance, solution, solution_idx):
    x, y = solution

    # Constraints (must not be violated)
    if 6*x + 9*y > 7000:  # Labor
        return 0
    if x + y > 1000:      # Chip sets
        return 0
    if 15*x + 10*y > 14000:  # Electronic components
        return 0
    if x < 0 or y < 0:    # Non-negativity
        return 0

    # Valid solution → return profit
    return 182 * x + 139 * y

# Gene space (can be restricted more precisely)
gene_space = [{'low': 0, 'high': 1000}, {'low': 0, 'high': 1000}]

# Create the GA instance
ga_instance = pygad.GA(
    num_generations=100,
    num_parents_mating=5,
    fitness_func=fitness_func,
    sol_per_pop=20,
    num_genes=2,
    gene_space=gene_space,
    mutation_percent_genes=10,
    mutation_type="random",
    crossover_type="single_point",
    stop_criteria=["reach_200000"]
)

# Run the GA
ga_instance.run()

# Output best result
solution, solution_fitness, _ = ga_instance.best_solution()
x_opt, y_opt = solution
print("Optimal solution found!")
print(f"Number of Pro tablets to produce: {x_opt:.2f}")
print(f"Number of Mini tablets to produce: {y_opt:.2f}")
print(f"Maximum profit: ${solution_fitness:.2f}")

Optimal solution found!
Number of Pro tablets to produce: 870.80
Number of Mini tablets to produce: 92.53
Maximum profit: $171348.33


In [1]:
import numpy as np
import pygad

# Problem dimensions
num_supplies = 4
num_demands = 4
num_vars = num_supplies * num_demands

# Supply and demand constraints
supply_capacity = np.array([50, 60, 30, 40])
demand_required = np.array([30, 50, 40, 60])

# Weighted value per unit to be maximized
weights = np.array([1, 2, 3, 4])
weight_matrix = np.tile(weights, (num_supplies, 1)).flatten()

# Feasibility matrix: 1 if allowed, 0 if not (shape 4x4)
feasibility_matrix = np.array([
    [1, 1, 1, 1],
    [1, 1, 0, 1],
    [1, 0, 0, 1],
    [1, 1, 1, 1]
])
feasibility_flat = feasibility_matrix.flatten()

# === Fitness Function ===
def fitness_func(solution, solution_idx):
    # Enforce feasibility: set disallowed variables to 0
    solution = np.where(feasibility_flat == 1, solution, 0)

    matrix = solution.reshape((num_supplies, num_demands))
    supply_used = matrix.sum(axis=1)
    demand_fulfilled = matrix.sum(axis=0)

    # Penalty terms
    supply_violation = np.sum(np.maximum(0, supply_used - supply_capacity))
    demand_violation = np.sum(np.abs(demand_fulfilled - demand_required))

    penalty = 1000 * (supply_violation + demand_violation)

    # Objective: weighted sum of demand fulfilled
    weighted_sum = np.sum(solution * weight_matrix)

    return weighted_sum - penalty

# === PyGAD Setup ===
ga_instance = pygad.GA(
    num_generations=200,
    num_parents_mating=10,
    fitness_func=fitness_func,
    sol_per_pop=20,
    num_genes=num_vars,
    init_range_low=0.0,
    init_range_high=50.0,
    gene_type=float,
    mutation_percent_genes=20,
    mutation_type="random",
    crossover_type="single_point"
)

# === Run Genetic Algorithm ===
ga_instance.run()

# === Results ===
best_solution, best_fitness, _ = ga_instance.best_solution()
best_matrix = best_solution.reshape((num_supplies, num_demands))
print("\n✅ Optimal Shipment Matrix (S1–S4 to D1–D4):")
print(np.round(best_matrix, 2))
print("\n🎯 Objective Function Value (weighted total demand fulfilled):", round(best_fitness, 2))

In PyGAD 2.20.0, the fitness function must accept 3 parameters:
1) The instance of the 'pygad.GA' class.
2) A solution to calculate its fitness value.
3) The solution's index within the population.

The passed fitness function named 'fitness_func' accepts 2 parameter(s).
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\envs\Lin_Opt\Lib\site-packages\pygad\pygad.py", line 960, in __init__
    raise ValueError(f"In PyGAD 2.20.0, the fitness function must accept 3 parameters:\n1) The instance of the 'pygad.GA' class.\n2) A solution to calculate its fitness value.\n3) The solution's index within the population.\n\nThe passed fitness function named '{fitness_func.__code__.co_name}' accepts {fitness_func.__code__.co_argcount} parameter(s).")
ValueError: In PyGAD 2.20.0, the fitness function must accept 3 parameters:
1) The instance of the 'pygad.GA' class.
2) A solution to calculate its fitness value.
3) The solution's index within the population.

The passed fitness funct

ValueError: In PyGAD 2.20.0, the fitness function must accept 3 parameters:
1) The instance of the 'pygad.GA' class.
2) A solution to calculate its fitness value.
3) The solution's index within the population.

The passed fitness function named 'fitness_func' accepts 2 parameter(s).

In [2]:
import numpy as np
import pygad

# Problem dimensions
num_supplies = 4
num_demands = 4
num_vars = num_supplies * num_demands

# Supply and demand
supply_capacity = np.array([50, 60, 30, 40])
demand_required = np.array([30, 50, 40, 60])

# Weighting from Excel: D1:1, D2:2, D3:3, D4:4
weights = np.array([1, 2, 3, 4])
weight_matrix = np.tile(weights, (num_supplies, 1)).flatten()

# Feasibility matrix (1 = allowed, 0 = disallowed)
feasibility_matrix = np.array([
    [1, 1, 1, 1],
    [1, 1, 0, 1],
    [1, 0, 0, 1],
    [1, 1, 1, 1]
])
feasibility_flat = feasibility_matrix.flatten()

# === Fitness Function (Correct Signature for PyGAD 2.20.0+) ===
def fitness_func(ga_instance, solution, solution_idx):
    # Apply feasibility mask
    solution = np.where(feasibility_flat == 1, solution, 0)

    matrix = solution.reshape((num_supplies, num_demands))
    supply_used = matrix.sum(axis=1)
    demand_fulfilled = matrix.sum(axis=0)

    # Constraint violations
    supply_violation = np.sum(np.maximum(0, supply_used - supply_capacity))
    demand_violation = np.sum(np.abs(demand_fulfilled - demand_required))

    # Penalty
    penalty = 1000 * (supply_violation + demand_violation)

    # Objective: maximize weighted total
    weighted_sum = np.sum(solution * weight_matrix)

    return weighted_sum - penalty

# === PyGAD Setup ===
ga_instance = pygad.GA(
    num_generations=200,
    num_parents_mating=10,
    fitness_func=fitness_func,
    sol_per_pop=20,
    num_genes=num_vars,
    init_range_low=0.0,
    init_range_high=60.0,
    gene_type=float,
    mutation_percent_genes=20,
    mutation_type="random",
    crossover_type="single_point"
)

# === Run the GA ===
ga_instance.run()

# === Results ===
best_solution, best_fitness, _ = ga_instance.best_solution()
best_matrix = best_solution.reshape((num_supplies, num_demands))

print("\n✅ Optimal Shipment Matrix (rounded):")
print(np.round(best_matrix, 2))
print("\n🎯 Objective Function Value (weighted total demand fulfilled):", round(best_fitness, 2))


✅ Optimal Shipment Matrix (rounded):
[[-5.070e+00 -2.570e+00  3.482e+01  2.316e+01]
 [ 3.000e-02  4.721e+01  1.543e+01  1.287e+01]
 [ 1.324e+01  4.453e+01  2.361e+01  1.667e+01]
 [ 2.182e+01  5.360e+00  5.070e+00  7.290e+00]]

🎯 Objective Function Value (weighted total demand fulfilled): -95.53


In [4]:
import numpy as np
import pygad

# Problem dimensions
num_supplies = 4
num_demands = 4
num_vars = num_supplies * num_demands

# Supply and demand
supply_capacity = np.array([50, 60, 30, 40])
demand_required = np.array([30, 50, 40, 60])

# Weighting from Excel: D1:1, D2:2, D3:3, D4:4
weights = np.array([1, 2, 3, 4])
weight_matrix = np.tile(weights, (num_supplies, 1)).flatten()

# Feasibility matrix (1 = allowed, 0 = disallowed)
feasibility_matrix = np.array([
    [1, 1, 1, 1],
    [1, 1, 0, 1],
    [1, 0, 0, 1],
    [1, 1, 1, 1]
])
feasibility_flat = feasibility_matrix.flatten()

# Set upper bound for genes (e.g., largest capacity or demand)
gene_upper_bound = max(supply_capacity.max(), demand_required.max())

# Constrain genes to [0, upper_bound]
gene_space = [{'low': 0.0, 'high': gene_upper_bound}] * num_vars

# === Fitness Function ===
def fitness_func(ga_instance, solution, solution_idx):
    # Mask disallowed variables
    solution = np.where(feasibility_flat == 1, solution, 0)

    matrix = solution.reshape((num_supplies, num_demands))
    supply_used = matrix.sum(axis=1)
    demand_fulfilled = matrix.sum(axis=0)

    # Constraint violations
    supply_violation = np.sum(np.maximum(0, supply_used - supply_capacity))
    demand_violation = np.sum(np.abs(demand_fulfilled - demand_required))

    penalty = 1000 * (supply_violation + demand_violation)

    weighted_sum = np.sum(solution * weight_matrix)
    return weighted_sum - penalty

# === Run GA ===
ga_instance = pygad.GA(
    num_generations=200,
    num_parents_mating=10,
    fitness_func=fitness_func,
    sol_per_pop=30,
    num_genes=num_vars,
    gene_space=gene_space,  # ✅ Constrain genes here
    gene_type=float,
    mutation_percent_genes=20,
    mutation_type="random",
    crossover_type="single_point"
)

ga_instance.run()

# === Results ===
best_solution, best_fitness, _ = ga_instance.best_solution()
best_matrix = best_solution.reshape((num_supplies, num_demands))

print("\n✅ Optimal Shipment Matrix (rounded, non-negative):")
print(np.round(best_matrix, 2))
print("\n🎯 Objective Function Value (weighted total demand fulfilled):", round(best_fitness, 2))


✅ Optimal Shipment Matrix (rounded, non-negative):
[[14.55  4.74 18.25 17.57]
 [11.02 38.76 43.31 11.02]
 [ 4.02 17.72  6.97 26.05]
 [ 0.45  4.32 21.71  4.74]]

🎯 Objective Function Value (weighted total demand fulfilled): -8394.68
